This notebook uses Julia to do symbolic computation: *it must run with a Julia kernel!*

Consider the problem of computing the slope of the tangent line to a point on a circle.

# 1. Implicit Differentiation with SymPy.jl

In [1]:
using SymPy

In [2]:
x = Sym("x")

x

In [3]:
y = SymFunction("y")

y

In [4]:
dy = diff(y(x), x)

d       
──(y(x))
dx      

In [5]:
typeof(dy)

Sym{PyCall.PyObject}

In [6]:
c = x^2 + y(x)^2 - 1

 2    2       
x  + y (x) - 1

In [7]:
typeof(c)

Sym{PyCall.PyObject}

In [8]:
dc = diff(c, x)

             d       
2⋅x + 2⋅y(x)⋅──(y(x))
             dx      

In [9]:
dydx = solve(dc, dy)

1-element Vector{Sym{PyCall.PyObject}}:
 -x/y(x)

In [10]:
typeof(dydx)

Vector{Sym{PyObject}} (alias for Array{Sym{PyCall.PyObject}, 1})

From the output above, we see that ``dydx`` is an array.  Arrays in Julia start at one.

In [11]:
symbolicslope = dydx[1]

-x  
────
y(x)

Let us now take a random point on the circle, which is generated by a random angle.

In [12]:
angle = 2*pi*rand()

3.0739308808121364

Because Julia is oriented towards numerical computing, the ``pi`` is a 64-bit floating point constant.

In [13]:
(a, b) = (cos(angle), sin(angle))

(-0.9977118154165967, 0.06761015735907423)

We store the coordinates of the point in a dictionary, which is a convenient data structure for substitution.

In [14]:
sd = Dict(x => a, y(x) => b)

Dict{Sym{PyCall.PyObject}, Float64} with 2 entries:
  y(x) => 0.0676102
  x    => -0.997712

In [15]:
slope = symbolicslope.subs(sd)

14.7568332095102

We see that the ``subs`` executed a simultaneous substitution.

Let us now do the slope computation via a Taylor series.  The ``series`` of ``SymPy`` works only for one variable, but it works well on purely symbolic expressions.  We declare new symbols ``X`` and ``Y`` for the variables in the circle equation and ``A`` and ``B`` for the coordinates of the point.

In [16]:
X, Y, A, B = Sym("X, Y, A, B")

(X, Y, A, B)

In [17]:
nc = X^2 + Y^2  - 1

 2    2    
X  + Y  - 1

In [18]:
cx = SymPy.series(nc, X, A, 2)

      2                   2    ⎛        2       ⎞
-1 + Y  + 2⋅A⋅(-A + X) + A  + O⎝(-A + X) ; X → A⎠

To convert the ``cx`` into a symbolic expression on which we can apply the ``series`` again, we select the coefficients and make the expression ``px``.

In [19]:
cx.coeff(X-A,1)

2⋅A

In [20]:
cx.coeff(X, 0)

 2    2    
A  + Y  - 1

In [21]:
px = cx.coeff(X, 0) + cx.coeff(X-A, 1)*(X-A)

 2                   2    
A  + 2⋅A⋅(-A + X) + Y  - 1

In [22]:
cy = series(px, Y, B, 2)

                     2                   2    ⎛        2       ⎞
-1 + 2⋅B⋅(-B + Y) + B  + 2⋅A⋅(-A + X) + A  + O⎝(-B + Y) ; Y → B⎠

In [23]:
py = cy.coeff(Y, 0) + cy.coeff(Y-B, 1)*(Y-B)

 2                   2                   
A  + 2⋅A⋅(-A + X) + B  + 2⋅B⋅(-B + Y) - 1

In [24]:
cY = py.subs(Dict(A=>a, B=>b))

-1.99542363083319⋅X + 0.135220314718148⋅Y - 2.0

In [25]:
- cY.coeff(X, 1)/cY.coeff(Y, 1)

14.7568332095102

In [26]:
slope

14.7568332095102

# 2. Implicit Differentiation via Symbolics.jl

In [27]:
using Symbolics

In [28]:
@variables x, y(x)

2-element Vector{Num}:
    x
 y(x)

Also ``SymPy`` has a ``Differential`` and in order not to confuse, because we have done ``using SymPy`` earlier in this notebook, we have have to declare that we are using the ``Differential`` from ``Symbolics``.

In [29]:
D = Symbolics.Differential(x)

Differential(x)

In [30]:
dy = D(y)

Differential(x)(y(x))

In [31]:
typeof(dy)

Num

In [32]:
c = x^2 + y^2 - 1

-1 + x^2 + y(x)^2

In [33]:
dc = D(c)

Differential(x)(-1 + x^2 + y(x)^2)

In [34]:
eq = expand_derivatives(dc)

2x + 2Differential(x)(y(x))*y(x)

In [35]:
SBslope = Symbolics.solve_for(eq, dy)

(-x) / y(x)

We have to make a new dictionary with the coordinates for the points, mapping them to the Symbolics variables.

In [36]:
nsd = Dict(x => a, y => b)

Dict{Num, Float64} with 2 entries:
  y(x) => 0.0676102
  x    => -0.997712

In [37]:
substitute(SBslope, nsd)

14.756833209510194

Let us compare with the slope we computed earlier.

In [38]:
slope

14.7568332095102